# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
driver=webdriver.Chrome()

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [4]:
name=driver.find_element_by_name('pht_lnm')
name.send_keys('Nguyen')

In [6]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_name('pht_status'))
select.select_by_visible_text('Cosmetologists')


In [7]:
button = driver.find_element_by_name('B1')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [59]:
results=driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    print('..........')
    print(result.text)

..........
Name and Location Order Basis for Order
..........
NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $500. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
..........
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
..........
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257 Date: 5/17/2018

Respondent is assessed an administrati

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [82]:
results=driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    names=result.find_elements_by_class_name('results_text')
    try:
        print(names[0].text)
    except:
        print("It didn't work")
  

It didn't work
NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [98]:
results=driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    description=result.find_elements_by_tag_name('td')
    try:
        print(description[2].text)
    except:
        print('nope')

nope
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implement

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [108]:
results=driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    names=result.find_elements_by_class_name('results_text')
    try:
        print(names[-2].text)
    except:
        print("It didn't work")
  

It didn't work
COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
COS20170023893


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [174]:
violations=[]
results=driver.find_elements_by_tag_name('tr')
for result in results[:10]:
    cosmetologists={}
    try:
        names=result.find_elements_by_class_name('results_text')
        description=result.find_elements_by_tag_name('td')


        cosmetologists['name']=names[0].text
        cosmetologists['violation_description']=description[2].text
        cosmetologists['violation_number']=names[-2].text
        cosmetologists['License_Number']=names[-3].text
        cosmetologists['ZIP_code']=names[3].text
        cosmetologists['county']=names[2].text
        cosmetologists['city']=names[1].text


        violations.append(cosmetologists)
    except:
        print("It didn't work")
        
    

It didn't work


### Save that to a CSV

- Tip: You'll want to use pandas here

In [178]:
import pandas as pd
%matplotlib inline
df=pd.DataFrame(violations)
df.to_csv("violations_cosmetologists", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [179]:
df.head()

,License_Number,ZIP_code,city,county,name,violation_description,violation_number
0,"780948, 1706491, 1699123",78217,SAN ANTONIO,BEXAR,"NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,COS20180004289
1,737708,79934,EL PASO,EL PASO,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,COS20180006594
2,731665,75604,LONGVIEW,GREGG,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,COS20180000257
3,"1347649, 760528",77014,HOUSTON,HARRIS,"NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",COS20180004915
4,767339,78255,SAN ANTONIO,BEXAR,"NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",COS20180009255
